# **Введение**

Всем привет, это моя тетрадь по дообучению с LORA, эта тетрадь собрана из многих моих тетрадей.
Здесь я постараюсь показать как я дошел до своего решения. Всем приятного чтения!

# **Штука для гугл коллаба**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Скучная, но необходимая часть - догрузка и  импорт Библиотек**

## **Догрузка**

In [2]:
!pip install -q transformers
!pip install -q peft
!pip install -q bitsandbytes
!pip install -q datasets --upgrade
!pip install -q accelerate
!pip install -q sentencepiece
!pip install -q torch
!pip install -q numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 100.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## **Библиотека**

In [3]:
import torch
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import json
from datasets import Dataset, DatasetDict
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig
from transformers import BitsAndBytesConfig
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel, PeftConfig

from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
import os # Для создания директории
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
import os # Для проверки пути
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
import os

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
import os

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training


from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
import os


import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
import os


import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
import os

## **Проверка GPU/CPU**

In [4]:
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    print(f"GPU доступен: {gpu_name}")
    device = torch.device("cuda")
else:
    print("GPU не доступен")
    device = torch.device("cpu")

GPU доступен: Tesla T4


# **Загрузка модели и токенизатора (с 4-битным квантованием)**

In [5]:
model_name = "sberbank-ai/rugpt3small_based_on_gpt2"
# model_name = "sberbank-ai/rugpt3medium_based_on_gpt2" - как показала практика буду использовать эту модель. но не суть

# Конфигурация для 4-битного квантования
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4", # Тип квантования - это база (часто используют nf4)
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True, # Использовать двойное квантование для еще большей экономии, экономим максимально
)

# Загрузка модели с квантованием
try:
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True #
    )
    print(f"Модель '{model_name}' успешно загружена с 4-битным квантованием ураааа")
except Exception as e:
    print(f"Ошибка при загрузке модели: {e}")
    print("Попробуем загрузить без явного torch_dtype=torch.bfloat16, если ошибка связана с ним")




###################################___Еще_одна_вариация_###########################################

    # Иногда на T4 bfloat16 может вызывать проблемы, попробуем с float16, поэтому попробуем след финт ушами
    bnb_config_fp16 = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16, # изменение только тут
        bnb_4bit_use_double_quant=True,
    )
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config_fp16,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True
    )
    print(f"Модель '{model_name}' успешно загружена с 4-битным квантованием и compute_dtype=torch.float16.")



######################################__________ТОКИНИЗАТОР___________###################################

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# Настройка токенизатора GPT-2 модели часто не имеют специального pad_token
# Мы можем использовать eos_token (end-of-sequence) в качестве pad_token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    print(f"Установлен pad_token: {tokenizer.pad_token} (был None, использован eos_token)")

# И уже по цепочке надо обновим конфигурацию модели, если pad_token_id не был установлен
if model.config.pad_token_id is None:
    model.config.pad_token_id = model.config.eos_token_id
    print(f"Установлен model.config.pad_token_id: {model.config.pad_token_id}")


############__Я_стараюсь_написать_ну_прям_очень_наглядный код,_поэтому_буду_выводить_все_в_этом_мире_########

print("\nИнформация о модели:")
print(f"  Количество параметров: {model.num_parameters() / 1e6:.2f}M (после квантования может быть меньше)") # num_parameters может не отражать фактический размер в памяти после квантования
print(f"  Конфигурация модели: {model.config.__class__.__name__}")

print("\nИнформация о токенизаторе:")
print(f"  Класс токенизатора: {tokenizer.__class__.__name__}")
print(f"  Размер словаря: {tokenizer.vocab_size}")
print(f"  Pad token: {tokenizer.pad_token} (ID: {tokenizer.pad_token_id})")
print(f"  EOS token: {tokenizer.eos_token} (ID: {tokenizer.eos_token_id})")
print(f"  BOS token: {tokenizer.bos_token} (ID: {tokenizer.bos_token_id})") # Обычно нет у GPT2

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/551M [00:00<?, ?B/s]

Модель 'sberbank-ai/rugpt3small_based_on_gpt2' успешно загружена с 4-битным квантованием ураааа


tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]


Информация о модели:
  Количество параметров: 125.23M (после квантования может быть меньше)
  Конфигурация модели: GPT2Config

Информация о токенизаторе:
  Класс токенизатора: GPT2TokenizerFast
  Размер словаря: 50257
  Pad token: <pad> (ID: 0)
  EOS token: </s> (ID: 2)
  BOS token: <s> (ID: 1)


# **Что у нас по данным, Джонии? Загрузка и просмотр данных ₍^. .^₎⟆**

In [6]:
file_path = "/content/drive/MyDrive/project/output.jsonl" # Путь к вашему файлу, пожалуйста, настройка под себя, укажите свой путь
data = []

try:
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))
    print(f"Успешно загружено {len(data)} записей из {file_path}")
except Exception as e:
    print(f"Ошибка при чтении файла {file_path}: {e}")

if data:
    dataset = Dataset.from_list(data)

    print("\nПервые 3 примера из датасета:")
    for i in range(min(3, len(dataset))):
        print(dataset[i])

    # Инфа о датасете
    print("\nИнформация о датасете:")
    print(dataset)
else:
    print("Данные не были загружены. Проверьте путь к файлу и его содержимое и прочитайте самый первый мой комментарий")

Успешно загружено 8641 записей из /content/drive/MyDrive/project/output.jsonl

Первые 3 примера из датасета:
{'instruction': 'Какую тему в интервью с Балакиной Татьяной Петровной обсудили в первую очередь?', 'input': '', 'output': 'Карьерный путь, науку и образование.'}
{'instruction': 'Какие направления в образовании упомянуты в контексте обсуждения с Татьяной Петровной?', 'input': '', 'output': 'Экономика и физика.'}
{'instruction': 'Почему важно участвовать в олимпиадах и хакатонах по мнению Татьяны Петровны?', 'input': '', 'output': 'Ответ на этот вопрос можно найти в интервью, но в данном тексте он не представлен.'}

Информация о датасете:
Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 8641
})


# **Форматирование данных и создание текстовых промптов**

In [7]:
# Проверка существования и заполненности датасета (да, в какой-то момент у меня были с этим проблемы)
if 'dataset' not in globals() or not dataset:
    print("Warning, исходный датасет не был загружен или пуст")
else:
    eos_token = tokenizer.eos_token  # Получаем eos_token из токенизатора
    if eos_token is None:
        print("Предупреждение: eos_token не найден, подумай почему")
        eos_token = ""

    def create_prompt_text(example):
        prompt_template = f"Инструкция: {example['instruction']}\n\nОтвет: {example['output']}{eos_token}"
        return {"text": prompt_template}

    # Применяем функцию ко всему датасету для создания новой колонки 'text'
    dataset_with_text_prompts = dataset.map(
        create_prompt_text,
        remove_columns=['instruction', 'input', 'output']
    )

    print("\nПервые 3 примера сгенерированных текстовых промптов")
    for i in range(min(3, len(dataset_with_text_prompts))):
        print(f"--- Пример {i+1} ---")
        print(dataset_with_text_prompts[i]['text'])
        print("--------------------")

    print("\nИнформация о датасете с текстовыми промптами:")
    print(dataset_with_text_prompts)
    if len(dataset_with_text_prompts) > 0:
        print("\nПример первой записи (ключи):")
        print(dataset_with_text_prompts[0].keys())
        print("\nТекст первой записи:")
        print(dataset_with_text_prompts[0]['text'])
    else:
        print("Датасет с текстовыми промптами пуст, проверяй где накосячил")

Map:   0%|          | 0/8641 [00:00<?, ? examples/s]


Первые 3 примера сгенерированных текстовых промптов
--- Пример 1 ---
Инструкция: Какую тему в интервью с Балакиной Татьяной Петровной обсудили в первую очередь?

Ответ: Карьерный путь, науку и образование.</s>
--------------------
--- Пример 2 ---
Инструкция: Какие направления в образовании упомянуты в контексте обсуждения с Татьяной Петровной?

Ответ: Экономика и физика.</s>
--------------------
--- Пример 3 ---
Инструкция: Почему важно участвовать в олимпиадах и хакатонах по мнению Татьяны Петровны?

Ответ: Ответ на этот вопрос можно найти в интервью, но в данном тексте он не представлен.</s>
--------------------

Информация о датасете с текстовыми промптами:
Dataset({
    features: ['text'],
    num_rows: 8641
})

Пример первой записи (ключи):
dict_keys(['text'])

Текст первой записи:
Инструкция: Какую тему в интервью с Балакиной Татьяной Петровной обсудили в первую очередь?

Ответ: Карьерный путь, науку и образование.</s>


# **Поработаем с токенизацией, далее поделим на train/val + установим формат**

In [8]:
#########################_Проверка_мол_все_ли_у_нас_хорошо_#########################

if 'dataset_with_text_prompts' not in globals() or not dataset_with_text_prompts:
    print("Ошибка: 'dataset_with_text_prompts' не найден, ищи ошибку")
else:
    def tokenize_function(examples):

        # Пояснительная бригада, что я натыкал и для чего

        # max_length можно будет подобрать позже, пока оставим значение по умолчанию токенизатора
        # или установим достаточно большое, чтобы вместить большинство наших промптов, но пока 512
        # truncation=True обрежет слишком длинные последовательности
        # padding="max_length" добавит паддинг до max_length


        return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

    # Токенизируем тексты
    tokenized_dataset_full = dataset_with_text_prompts.map(tokenize_function, batched=True, remove_columns=["text"])
    print("\nИнформация о полном токенизированном датасете до разделения на train/val")
    print(tokenized_dataset_full)

#########################_Делим_данные_на_train/val_#########################
    try:
        split_dataset = tokenized_dataset_full.train_test_split(test_size=0.1, seed=42)
        train_dataset = split_dataset['train']
        eval_dataset = split_dataset['test']
        train_dataset.set_format("torch")
        eval_dataset.set_format("torch")

        print("\nДатасет успешно разделен train/val")
        print(f"Размер train: {len(train_dataset)}")
        print(f"Размер val: {len(eval_dataset)}")


    except Exception as e:
        print(f"Произошла ошибка, иди исправляй: {e}")

Map:   0%|          | 0/8641 [00:00<?, ? examples/s]


Информация о полном токенизированном датасете до разделения на train/val
Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 8641
})

Датасет успешно разделен train/val
Размер train: 7776
Размер val: 865


# **Конфигурация LoRA и QLoRA**

# **Подготовка модели к PEFT и конфигурация LoRA**

In [9]:
#########################_Проверка_мол_все_ли_у_нас_хорошо_#########################

if 'model' not in globals() or 'tokenizer' not in globals():
    print("Warning: Модель или токенизатор не загружены, ищи ошибку")
elif 'train_dataset' not in globals() or 'eval_dataset' not in globals():
    print("Warning: Обучающий или валидационный датасет не готовы")
else:
    print("Модель, токенизатор и датасеты готовы")

#########################_Подготовка модели для k-битного обучения_#########################

    # Это необходимо для стабильности при использовании QLoRA
    # Добавляет необходимые хуки и изменяет некоторые слои для совместимости с LoRA поверх квантованных весов
    try:
        model = prepare_model_for_kbit_training(model)
        print("Модель подготовлена для k-bit training QLoRA")
    except Exception as e:
        print(f"Ошибка при prepare_model_for_kbit_training: {e}")
        print("Продолжаем без этого шага, но могут быть проблемы со стабильностью QLoRA, так что думайте")

#########################_Определение_конфигурации_LoRA_#########################

    lora_config = LoraConfig(
        r=16,  # Ранг LoRA-матриц, больше параметров, потенциально лучше, но дольше обучение.
        lora_alpha=32, # Альфа для масштабирования LoRA
        target_modules=[
            "c_attn", # Для GPT-2 это слои query, key, value в self-attention.
                      # Для других моделей могут быть другие названия, например, "q_proj", "v_proj", "k_proj"
            # "c_proj", # Можно добавить и другие линейные слои, если необходимо
            # "c_fc"    # Слой в feed-forward сети
        ],
        lora_dropout=0.05,
        bias="none", # "none" обычно хорошо работает
        task_type="CAUSAL_LM" # Тип задачи
    )
    print("\nLoraConfig создана:")
    print(lora_config)


#########################_Применение_LoRA_к_модели_#########################

    try:
        lora_model = get_peft_model(model, lora_config)
        print("\nLoRA применена, красава")
        print("\nИнформация о параметрах после применения LoRA")
        lora_model.print_trainable_parameters()
        model_to_train = lora_model

    except Exception as e:
        print(f"Ошибка при применении LoRA к модели: {e}")
        model_to_train = None

    # Проверим, что модель готова для обучения
    if model_to_train:
        print("\nМодель готова к обучению с LoRA")
    else:
        print("\nМодель НЕ готова к обучению, чек ошибки выше")

Модель, токенизатор и датасеты готовы
Модель подготовлена для k-bit training QLoRA

LoraConfig создана:
LoraConfig(task_type='CAUSAL_LM', peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, inference_mode=False, r=16, target_modules={'c_attn'}, exclude_modules=None, lora_alpha=32, lora_dropout=0.05, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False)

LoRA применена, красава

Информация о параметрах после применения LoRA
trainable params: 589,824 || all params: 125,821,440 || trainable%: 0.4688

Модель готова к обучению с LoRA


Обучение (Fine-tuning).


# **Настройка аргументов обучения + запуск Trainer**

In [10]:
#########################_Проверка_мол_все_ли_у_нас_хорошо_#########################

if 'model_to_train' not in globals() or not model_to_train:
    print("Warning: Модель для обучения model_to_train не определена, ищи ошибку")
elif 'train_dataset' not in globals() or 'eval_dataset' not in globals():
    print("Warnin: Обучающий или валидационный датасет не готовы")
elif 'tokenizer' not in globals():
    print("Warnin: Токенизатор не загружен")
else:
    print("Все компоненты для обучения готовы, красава")

    # Определяем директорию для сохранения результатов
    output_dir = "./results_rugpt3small_lora_qa" # пожалуйста, пишите свою, тут моя


#########################_____Аргументы___обучения_____#########################

    # Аргументы обучения для transformers версии ~4.41.3
    # Учитываем ошибку ValueError: --load_best_model_at_end requires the save and eval strategy to match, мы совершенствуемся
    training_args = TrainingArguments(
        output_dir="./results_rugpt3small_lora_qa_epoch1_lr1e-4", # Новая папка для результатов, пишите свою, тут моя
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        num_train_epochs=1,
        learning_rate=1e-4,
        logging_strategy="steps",
        logging_steps=50,
        save_strategy="steps",
        save_steps=100,
        eval_strategy="steps",
        eval_steps=100,
        do_eval=True,
        save_total_limit=2,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        report_to="none",
        fp16=True,
    )


#########################_Этот_блок_отвечает_за_динамическое_формирование_батчей_данных_во_время_обучения_#########################

    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

#########################_Создание_Trainer_и_обучение_#################

    trainer = Trainer(
        model=model_to_train,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
    )

    print("\nTrainer сконфигурирован, начинаем обучение")


    try:
        train_result = trainer.train()
        print("\nОбучение завершено!")

        final_model_path = f"{output_dir}/final_model_adapters"
        trainer.save_model(final_model_path)
        print(f"Финальные LoRA адаптеры сохранены в: {final_model_path}")

        metrics = train_result.metrics
        print("\nМетрики обучения:")
        print(metrics)
        print("\nЛог обучения (история):")
        for log_entry in trainer.state.log_history:
            print(log_entry)

    except Exception as e:
        print(f"\nПроизошла ошибка во время обучения: {e}")
        import traceback
        traceback.print_exc()

<ipython-input-10-645fbef4aa4f>:48: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Все компоненты для обучения готовы, красава

Trainer сконфигурирован, начинаем обучение


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss
100,4.835100,3.684391
200,3.519200,3.085459
300,3.150300,2.874985
400,2.988400,2.734653


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/


Обучение завершено!
Финальные LoRA адаптеры сохранены в: ./results_rugpt3small_lora_qa/final_model_adapters

Метрики обучения:
{'train_runtime': 611.6764, 'train_samples_per_second': 12.713, 'train_steps_per_second': 0.795, 'total_flos': 2045896481046528.0, 'train_loss': 3.8169460375122575, 'epoch': 1.0}

Лог обучения (история):
{'loss': 7.3757, 'grad_norm': 2.6292386054992676, 'learning_rate': 9.032921810699589e-05, 'epoch': 0.102880658436214, 'step': 50}
{'loss': 4.8351, 'grad_norm': 2.7803878784179688, 'learning_rate': 8.004115226337449e-05, 'epoch': 0.205761316872428, 'step': 100}
{'eval_loss': 3.6843907833099365, 'eval_runtime': 18.0599, 'eval_samples_per_second': 47.896, 'eval_steps_per_second': 6.035, 'epoch': 0.205761316872428, 'step': 100}
{'loss': 3.8748, 'grad_norm': 2.3525454998016357, 'learning_rate': 6.975308641975309e-05, 'epoch': 0.30864197530864196, 'step': 150}
{'loss': 3.5192, 'grad_norm': 1.8626518249511719, 'learning_rate': 5.9465020576131694e-05, 'epoch': 0.41152

# **Тестирование Дообученной Модели в Colab**

Загрузка модели с LoRA адаптерами и тестовая генерация

In [19]:
#########################_Проверка_мол_все_ли_у_нас_хорошо_#########################

if 'model_name' not in globals():
    model_name = "sberbank-ai/rugpt3small_based_on_gpt2"
if 'tokenizer' not in globals() or tokenizer.name_or_path != model_name:
    print(f"Загрузка токенизатора для {model_name}...")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        print(f"Установлен pad_token: {tokenizer.pad_token}")


# Путь к сохраненным LoRA адаптерам
lora_adapters_path = "./results_rugpt3small_lora_qa/final_model_adapters"

#########################_Загрузка_конфигурации_PEFT_LoRA_#########################

try:
    config = PeftConfig.from_pretrained(lora_adapters_path)
    print("PeftConfig успешно загружен.")
except Exception as e:
    print(f"Ошибка при загрузке PeftConfig: {e}. Проверьте путь: {lora_adapters_path}")
    config = None


#########################_Загрузка_базовой_модели_(неквантованной/квантованной, как при обучении)#########################
if config:

    # Для инференса мы можем загрузить ее без BitsAndBytesConfig, если хотим максимального качества
    # или с BitsAndBytesConfig, если хотим сохранить экономию памяти
    # Летс, чек с BitsAndBytesConfig для консистентности и экономии

    bnb_config_inference = BitsAndBytesConfig(
        load_in_4bit=True, # Или load_in_8bit=True
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16, # для T4 обычно хорошо
        bnb_4bit_use_double_quant=False, # Можно попробовать True для большей экономии
    )

    print(f"Загрузка базовой модели '{config.base_model_name_or_path}' с квантованием для инференса")
    try:
        base_model = AutoModelForCausalLM.from_pretrained(
            config.base_model_name_or_path,
            quantization_config=bnb_config_inference, # Применяем квантование
            device_map="auto",
            trust_remote_code=True
        )
        print("Базовая модель успешно загружена с квантованием, красава")


#########################_Слияние" LoRA адаптеров с базовой моделью_#########################
        print(f"Загрузка модели с LoRA адаптерами из {lora_adapters_path}...")
        inference_model = PeftModel.from_pretrained(base_model, lora_adapters_path)

        # Если мы хотим получить "слитую" модель (где веса LoRA объединены с базовыми)
        # и использовать ее как обычную модель transformers (это требует больше памяти)
        # merged_model = inference_model.merge_and_unload()
        # print("Модель слита (LoRA адаптеры интегрированы в базовую модель).")
        # final_model_for_inference = merged_model

        # Пока будем использовать PeftModel напрямую, это более экономно по памяти
        final_model_for_inference = inference_model
        final_model_for_inference.eval() # Переводим модель в режим оценки

        print("Модель с LoRA адаптерами готова для инференса, красава")

    except Exception as e:
        print(f"Ошибка при загрузке базовой модели или применении LoRA: {e}")
        final_model_for_inference = None
else:
    final_model_for_inference = None

#########################_Функция_для_генерации_ответа_#########################
if final_model_for_inference and tokenizer:
    def generate_answer(question, model, tokenizer, max_new_tokens=100, temperature=0.7, top_p=0.9):
        # Формируем промпт так же, как при обучении (но без ответа)
        prompt = f"Инструкция: {question}\n\nОтвет:"

        inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=512-max_new_tokens)
        inputs = {k: v.to(next(model.parameters()).device) for k, v in inputs.items()}


        # Генерация ответа
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                temperature=temperature,
                top_p=top_p,
                # num_beams=1,
                do_sample=True,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id
            )

        # Декодируем сгенерированные токены в текст (Мы берем только сгенерированную часть, исключая промпт)
        answer_tokens = outputs[0][inputs['input_ids'].shape[1]:]
        answer_text = tokenizer.decode(answer_tokens, skip_special_tokens=True)

        return answer_text.strip()

    # Тестовые запросы, взял из датасета output.jsonl
    test_questions = [
        "Кто такой Никита Чухров?",
        "Кто такой Филипп Ульянкин?",
        "Кто по образованию Балакина Татьяна Петровна?",
        "Какие направления в образовании упомянуты в контексте обсуждения с Татьяной Петровной?",
        "Почему важно участвовать в олимпиадах и хакатонах по мнению Татьяны Петровны?" ]

    print("\n---###_____Тестирование_модели_____###---")
    for q in test_questions:
        print(f"\nВопрос: {q}")
        answer = generate_answer(q, final_model_for_inference, tokenizer)
        print(f"Ответ модели: {answer}")

else:
    print("Модель для инференса не была загружена, ищи ошибки")

PeftConfig успешно загружен.
Загрузка базовой модели 'sberbank-ai/rugpt3small_based_on_gpt2' с квантованием для инференса
Базовая модель успешно загружена с квантованием, красава
Загрузка модели с LoRA адаптерами из ./results_rugpt3small_lora_qa/final_model_adapters...
Модель с LoRA адаптерами готова для инференса, красава

---###_____Тестирование_модели_____###---

Вопрос: Кто такой Никита Чухров?
Ответ модели: Ответ: Ответ: Никита Чухров – автор проекта «Мастер-класс».
„Мастер-класс“ – это проект, цель которого – помочь людям разобраться в себе, научиться принимать решения, ставить цели и достигать их, помогать людям достигать их, помогать людям достигать их, помогать людям достигать их, помогать людям достигать их, помогать людям достигать их, помогать людям достигать их, помогать людям достигать их, помогать людям достигать их, помогать людям достигать их, помогать людям достигать их, помогать людям достигать их,

Вопрос: Кто такой Филипп Ульянкин?
Ответ модели: Филипп Ульянкин — э

In [ ]:
#########################_Проверка_мол_все_ли_у_нас_хорошо_#########################

if 'model_to_train' not in globals() or not model_to_train:
    print("WARNING: Модель для обучения model_to_train не определена")
    raise RuntimeError("model_to_train не определена, Прерываем принудительно")
elif 'train_dataset' not in globals() or 'eval_dataset' not in globals():
    print("WARNING: train/val датасет не готовы")
    raise RuntimeError("train_dataset или eval_dataset не определены, Прерываем принудительно")

elif 'tokenizer' not in globals():
    print("WARNING: Токенизатор не загружен")
    raise RuntimeError("tokenizer не определен, Прерываем принудительно")
else:
    print("Все компоненты для обучения готовы")

    current_output_dir = "./results_rugpt3small_lora_qa_epoch1_lr2e-4" # пожалуйста, указывайте свой путь
    print(f"Результаты будут сохранены в: {current_output_dir}")


    training_args = TrainingArguments(
        output_dir=current_output_dir,
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        num_train_epochs=1,
        learning_rate=2e-4,
        logging_strategy="steps",
        logging_steps=50,
        save_strategy="steps",
        save_steps=100,
        eval_strategy="steps",
        eval_steps=100,
        do_eval=True,
        save_total_limit=2,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        report_to="none",
        fp16=True,
    )

    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

    # Создание Trainer

    if not hasattr(model_to_train, 'peft_config'):
        print("ПРЕДУПРЕЖДЕНИЕ: 'model_to_train' не выглядит как PEFT модель.")

    trainer = Trainer(
        model=model_to_train,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
    )

#########################_Обучение_#########################

    try:
        train_result = trainer.train()
        print("\nОбучение завершено!")
        final_lora_adapters_path = f"{current_output_dir}/final_lora_adapters"
        trainer.save_model(final_lora_adapters_path)
        print(f"Финальные LoRA адаптеры сохранены в: {final_lora_adapters_path}")

        metrics = train_result.metrics
        print("\nМетрики обучения:")
        print(metrics)
        print("\nЛог обучения (история):")
        for log_entry in trainer.state.log_history:
            print(log_entry)

    except Exception as e:
        print(f"\nПроизошла ошибка во время обучения, переделывай: {e}")
        import traceback
        traceback.print_exc()

<ipython-input-21-5fd7901eaf7c>:48: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Все компоненты для обучения готовы
Результаты будут сохранены в: ./results_rugpt3small_lora_qa_epoch1_lr2e-4


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss
100,2.806100,2.598723
200,2.715700,2.543581
300,2.666000,2.514507
400,2.633600,2.496433


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/


Обучение завершено!
Финальные LoRA адаптеры сохранены в: ./results_rugpt3small_lora_qa_epoch1_lr2e-4/final_lora_adapters

Метрики обучения:
{'train_runtime': 617.1397, 'train_samples_per_second': 12.6, 'train_steps_per_second': 0.788, 'total_flos': 2045896481046528.0, 'train_loss': 2.7035809112674416, 'epoch': 1.0}

Лог обучения (история):
{'loss': 2.8812, 'grad_norm': 1.534956693649292, 'learning_rate': 0.00017983539094650205, 'epoch': 0.102880658436214, 'step': 50}
{'loss': 2.8061, 'grad_norm': 1.4701067209243774, 'learning_rate': 0.00015925925925925927, 'epoch': 0.205761316872428, 'step': 100}
{'eval_loss': 2.5987231731414795, 'eval_runtime': 18.1778, 'eval_samples_per_second': 47.585, 'eval_steps_per_second': 5.996, 'epoch': 0.205761316872428, 'step': 100}
{'loss': 2.74, 'grad_norm': 2.026113986968994, 'learning_rate': 0.00013868312757201647, 'epoch': 0.30864197530864196, 'step': 150}
{'loss': 2.7157, 'grad_norm': 1.4636681079864502, 'learning_rate': 0.00011810699588477368, 'epoch

In [ ]:
lora_adapters_path = "./results_rugpt3small_lora_qa_epoch1_lr2e-4/final_lora_adapters"
base_model_name_for_inference = "sberbank-ai/rugpt3small_based_on_gpt2"

# Параметры генерации
generation_max_new_tokens = 150
generation_temperature = 0.6 # Можно начать с этого, потом экспериментировать
generation_top_p = 0.9

print(f"Загрузка токенизатора для: {base_model_name_for_inference}")
try:
    tokenizer = AutoTokenizer.from_pretrained(base_model_name_for_inference)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        print(f"Установлен pad_token: {tokenizer.pad_token} (ID: {tokenizer.pad_token_id})")
except Exception as e:
    print(f"Ошибка при загрузке токенизатора: {e}")
    tokenizer = None


#########################_Загрузка_конфигурации_PEFT_(LoRA)_#########################

if tokenizer:
    try:
        print(f"\nЗагрузка PeftConfig из: {lora_adapters_path}")
        config = PeftConfig.from_pretrained(lora_adapters_path)
        if config.base_model_name_or_path != base_model_name_for_inference:
            print(f"ПРЕДУПРЕЖДЕНИЕ: Имя базовой модели в PeftConfig ({config.base_model_name_or_path}) "
                  f"не совпадает с ожидаемым ({base_model_name_for_inference}). Используем имя из PeftConfig.")
        actual_base_model_name = config.base_model_name_or_path
        print("PeftConfig успешно загружен")
    except Exception as e:
        print(f"Ошибка при загрузке PeftConfig: {e}. Проверьте путь: {lora_adapters_path}")
        config = None
        actual_base_model_name = None
else:
    config = None
    actual_base_model_name = None

final_model_for_inference = None
if config and actual_base_model_name:

#########################_Загрузка базовой модели с квантованием_#########################

    bnb_config_inference = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=False,
    )

    print(f"\nЗагрузка базовой модели '{actual_base_model_name}' с квантованием для инференса...")
    try:
        base_model = AutoModelForCausalLM.from_pretrained(
            actual_base_model_name,
            quantization_config=bnb_config_inference,
            device_map="auto",
            trust_remote_code=True
        )
        print("Базовая модель успешно загружена с квантованием")

#########################_Применение LoRA адаптеров_#########################

        print(f"Загрузка и применение LoRA адаптеров из {lora_adapters_path} к базовой модели...")
        final_model_for_inference = PeftModel.from_pretrained(base_model, lora_adapters_path)
        final_model_for_inference.eval() # Переводим модель в режим оценки
        print("Модель с LoRA адаптерами готова для инференса")

    except Exception as e:
        print(f"Ошибка при загрузке базовой модели или применении LoRA: {e}")
        final_model_for_inference = None
else:
    if not tokenizer:
        print("Токенизатор не был загружен, инференс невозможен.")
    elif not config:
         print("PeftConfig не был загружен, инференс невозможен.")
    elif not actual_base_model_name:
        print("Не удалось определить имя базовой модели из PeftConfig.")

#########################_Функция_для_генерации_ответа__#########################
if final_model_for_inference and tokenizer:
    def generate_answer(question, model, tokenizer, max_new_tokens, temperature, top_p):
        prompt = f"Инструкция: {question}\n\nОтвет:"
        inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=512 - max_new_tokens)
        inputs = {k: v.to(next(model.parameters()).device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                temperature=temperature,
                top_p=top_p,
                do_sample=True,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id
            )

        answer_tokens = outputs[0][inputs['input_ids'].shape[1]:]
        answer_text = tokenizer.decode(answer_tokens, skip_special_tokens=True)

        # Постобработка для удаления лишних "Ответ:"
        processed_answer = answer_text.strip()
        while processed_answer.lower().startswith("ответ:"):
            processed_answer = processed_answer[len("ответ:"):]
            processed_answer = processed_answer.strip()

        return processed_answer

    # Тестовые запросы
    test_questions = [
        "Кто по образованию Балакина Татьяна Петровна?",
        "Какие направления в образовании упомянуты в контексте обсуждения с Татьяной Петровной?",
        "Почему важно участвовать в олимпиадах и хакатонах по мнению Татьяны Петровны?"
    ]

    print("\n--- Тестирование модели ---")
    print(f"Используются адаптеры из: {lora_adapters_path}")
    print(f"Параметры генерации: max_new_tokens={generation_max_new_tokens}, temperature={generation_temperature}, top_p={generation_top_p}")
    for q in test_questions:
        print(f"\nВопрос: {q}")
        answer = generate_answer(q, final_model_for_inference, tokenizer,
                                 generation_max_new_tokens, generation_temperature, generation_top_p)
        print(f"Ответ модели: {answer}")

else:
    print("\nМодель для инференса не была загружена или токенизатор отсутствует. Ищи ошибки.")

Загрузка токенизатора для: sberbank-ai/rugpt3small_based_on_gpt2

Загрузка PeftConfig из: ./results_rugpt3small_lora_qa_epoch1_lr2e-4/final_lora_adapters
PeftConfig успешно загружен.

Загрузка базовой модели 'sberbank-ai/rugpt3small_based_on_gpt2' с квантованием для инференса...
Базовая модель успешно загружена с квантованием.
Загрузка и применение LoRA адаптеров из ./results_rugpt3small_lora_qa_epoch1_lr2e-4/final_lora_adapters к базовой модели...
Модель с LoRA адаптерами готова для инференса.

--- Тестирование модели ---
Используются адаптеры из: ./results_rugpt3small_lora_qa_epoch1_lr2e-4/final_lora_adapters
Параметры генерации: max_new_tokens=150, temperature=0.6, top_p=0.9

Вопрос: Кто по образованию Балакина Татьяна Петровна?
Ответ модели: Балакина Татьяна Петровна — преподаватель, кандидат филологических наук, автор многочисленных публикаций, в том числе нескольких монографий.

Вопрос: Какие направления в образовании упомянуты в контексте обсуждения с Татьяной Петровной?
Ответ м

# **Переобучение с lr=1e-4, 1 эпоха, с сохранением в подпапку**

In [ ]:
#########################_Проверка_мол_все_ли_у_нас_хорошо_#########################

if 'model_to_train' not in globals() or not model_to_train:
    print("WARNING: Модель для обучения model_to_train не определена")
    raise RuntimeError("model_to_train не определена, принудительное прерывание.")
elif 'train_dataset' not in globals() or 'eval_dataset' not in globals():
    print("WARNING: train/val датасет не готовы")
    raise RuntimeError("train_dataset или eval_dataset не определены, принудительное прерывание.")
elif 'tokenizer' not in globals():
    print("WARNING: Токенизатор не загружен")
    raise RuntimeError("tokenizer не определен. принудительное прерывание")
else:
    print("Все компоненты для обучения готовы.")

    base_output_dir = "./results_rugpt3small_lora_qa_epoch1_lr1e-4_v2" # Добавил _v2 для уникальности
    # Путь для сохранения финальных адаптеров внутри base_output_dir
    final_lora_adapters_path = os.path.join(base_output_dir, "final_lora_adapters")

    print(f"Результаты (чекпоинты) будут в: {base_output_dir}")
    print(f"Финальные адаптеры будут сохранены в: {final_lora_adapters_path}")

    # Аргументы обучения
    training_args = TrainingArguments(
        output_dir=base_output_dir,         # Чекпоинты будут сохраняться сюда
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        num_train_epochs=1,
        learning_rate=1e-4,
        logging_strategy="steps",
        logging_steps=50,
        save_strategy="steps",
        save_steps=100,
        eval_strategy="steps",
        eval_steps=100,
        do_eval=True,
        save_total_limit=2,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        report_to="none",
        fp16=True,
    )

    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

    if not hasattr(model_to_train, 'peft_config'):
        print("ПРЕДУПРЕЖДЕНИЕ: 'model_to_train' не выглядит как PEFT модель.")

    trainer = Trainer(
        model=model_to_train,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
    )

    print("\nTrainer сконфигурирован. Начинаем обучение...")

    try:
        train_result = trainer.train()
        print("\nОбучение завершено!")

        # Явно создаем директорию для финальных адаптеров, если ее нет
        os.makedirs(final_lora_adapters_path, exist_ok=True)
        trainer.save_model(final_lora_adapters_path)
        print(f"Финальные LoRA адаптеры сохранены в: {final_lora_adapters_path}")

        # tokenizer.save_pretrained(final_lora_adapters_path) # Если нужно

        metrics = train_result.metrics
        print("\nМетрики обучения:")
        print(metrics)
        print("\nЛог обучения (история):")
        for log_entry in trainer.state.log_history:
            print(log_entry)

    except Exception as e:
        print(f"\nПроизошла ошибка во время обучения: {e}")
        import traceback
        traceback.print_exc()

<ipython-input-23-e21b97bdab94>:63: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Все компоненты для обучения готовы.
Результаты (чекпоинты) будут в: ./results_rugpt3small_lora_qa_epoch1_lr1e-4_v2
Финальные адаптеры будут сохранены в: ./results_rugpt3small_lora_qa_epoch1_lr1e-4_v2/final_lora_adapters

Trainer сконфигурирован. Начинаем обучение...


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss
100,2.702000,2.531617
200,2.655200,2.510149
300,2.632900,2.496098
400,2.618900,2.489911


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/


Обучение завершено!
Финальные LoRA адаптеры сохранены в: ./results_rugpt3small_lora_qa_epoch1_lr1e-4_v2/final_lora_adapters

Метрики обучения:
{'train_runtime': 612.0398, 'train_samples_per_second': 12.705, 'train_steps_per_second': 0.794, 'total_flos': 2045896481046528.0, 'train_loss': 2.6483464182159047, 'epoch': 1.0}

Лог обучения (история):
{'loss': 2.7115, 'grad_norm': 1.3129390478134155, 'learning_rate': 8.991769547325102e-05, 'epoch': 0.102880658436214, 'step': 50}
{'loss': 2.702, 'grad_norm': 1.419616937637329, 'learning_rate': 7.962962962962964e-05, 'epoch': 0.205761316872428, 'step': 100}
{'eval_loss': 2.5316171646118164, 'eval_runtime': 18.2369, 'eval_samples_per_second': 47.431, 'eval_steps_per_second': 5.977, 'epoch': 0.205761316872428, 'step': 100}
{'loss': 2.6608, 'grad_norm': 1.4007493257522583, 'learning_rate': 6.934156378600823e-05, 'epoch': 0.30864197530864196, 'step': 150}
{'loss': 2.6552, 'grad_norm': 1.2709611654281616, 'learning_rate': 5.905349794238684e-05, 'ep

# **Новый блок, чтобы не заплывало в глазах, новый тест**

In [ ]:
lora_adapters_path = "/content/results_rugpt3small_lora_qa_epoch1_lr1e-4_v2/final_lora_adapters"


base_model_name_for_inference = "sberbank-ai/rugpt3small_based_on_gpt2"

# Набор 1: С repetition_penalty и умеренной температурой
generation_params_set = {
    "max_new_tokens": 150,
    "temperature": 0.1, # Можно варьировать 0.2 - 0.7
    "top_p": 0.9,
    "repetition_penalty": 1.15,
    "do_sample": True
}

# Набор 2: Beam Search (альтернатива)
# generation_params_set = {
#     "max_new_tokens": 150,
#     "num_beams": 3,
#     "early_stopping": True,
#     "repetition_penalty": 1.1,
#     "do_sample": False
# }

current_generation_params = generation_params_set
print(f"Загрузка токенизатора для: {base_model_name_for_inference}")

try:
    tokenizer = AutoTokenizer.from_pretrained(base_model_name_for_inference)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        print(f"Установлен pad_token: {tokenizer.pad_token} (ID: {tokenizer.pad_token_id})")
except Exception as e:
    print(f"Ошибка при загрузке токенизатора: {e}")
    tokenizer = None

if not os.path.exists(lora_adapters_path):
    print(f"ОШИБКА: Путь к адаптерам не найден: {lora_adapters_path}")
    print("Пожалуйста, проверьте переменную 'lora_adapters_path' и убедитесь, что адаптеры существуют по этому пути")
    config = None
    actual_base_model_name = None
elif tokenizer:
    try:
        print(f"\nЗагрузка PeftConfig из: {lora_adapters_path}")
        config = PeftConfig.from_pretrained(lora_adapters_path)
        if config.base_model_name_or_path.lower() != base_model_name_for_inference.lower():
            print(f"ПРЕДУПРЕЖДЕНИЕ: Имя базовой модели в PeftConfig ({config.base_model_name_or_path}) "
                  f"не совпадает с ожидаемым ({base_model_name_for_inference}). Используем имя из PeftConfig.")
        actual_base_model_name = config.base_model_name_or_path
        print("PeftConfig успешно загружен.")
    except Exception as e:
        print(f"Ошибка при загрузке PeftConfig: {e}")
        config = None
        actual_base_model_name = None
else:
    config = None
    actual_base_model_name = None


final_model_for_inference = None
if config and actual_base_model_name:
    bnb_config_inference = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=False,
    )

    print(f"\nЗагрузка базовой модели '{actual_base_model_name}' с квантованием для инференса...")
    try:
        base_model = AutoModelForCausalLM.from_pretrained(
            actual_base_model_name,
            quantization_config=bnb_config_inference,
            device_map="auto",
            trust_remote_code=True
        )
        print("Базовая модель успешно загружена с квантованием.")

        print(f"Загрузка и применение LoRA адаптеров из {lora_adapters_path} к базовой модели...")
        final_model_for_inference = PeftModel.from_pretrained(base_model, lora_adapters_path)
        final_model_for_inference.eval()
        print("Модель с LoRA адаптерами готова для инференса.")

    except Exception as e:
        print(f"Ошибка при загрузке базовой модели или применении LoRA: {e}")
        final_model_for_inference = None
else:
    if not tokenizer:
        print("Токенизатор не был загружен")
    elif not os.path.exists(lora_adapters_path):
        pass
    elif not config:
         print("PeftConfig не был загружен")
    elif not actual_base_model_name:
        print("Не удалось определить имя базовой модели из PeftConfig.")


if final_model_for_inference and tokenizer:
    def generate_answer(question, model, tokenizer, gen_params):
        prompt = f"Инструкция: {question}\n\nОтвет:"
        max_input_length = 512 - gen_params.get("max_new_tokens", 100)
        inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=max_input_length)
        inputs = {k: v.to(next(model.parameters()).device) for k, v in inputs.items()}

        generation_call_params = {**inputs, **gen_params}
        generation_call_params['pad_token_id'] = tokenizer.pad_token_id
        generation_call_params['eos_token_id'] = tokenizer.eos_token_id

        with torch.no_grad():
            outputs = model.generate(**generation_call_params)

        answer_tokens = outputs[0][inputs['input_ids'].shape[1]:]
        answer_text = tokenizer.decode(answer_tokens, skip_special_tokens=True)

        processed_answer = answer_text.strip()
        while processed_answer.lower().startswith("ответ:"):
            processed_answer = processed_answer[len("ответ:"):]
            processed_answer = processed_answer.strip()

        return processed_answer

    test_questions = [
        "Кто по образованию Балакина Татьяна Петровна?",
        "Какие направления в образовании упомянуты в контексте обсуждения с Татьяной Петровной?",
        "Почему важно участвовать в олимпиадах и хакатонах по мнению Татьяны Петровны?"
    ]

    print("\n--- Тестирование модели ---")
    print(f"Используются адаптеры из: {lora_adapters_path}")
    print(f"Используемые параметры генерации: {current_generation_params}")
    for q in test_questions:
        print(f"\nВопрос: {q}")
        answer = generate_answer(q, final_model_for_inference, tokenizer, current_generation_params)
        print(f"Ответ модели: {answer}")
else:
    print("\nМодель для инференса не была загружена или токенизатор отсутствует. Проверьте ошибки выше.")

Загрузка токенизатора для: sberbank-ai/rugpt3small_based_on_gpt2

Загрузка PeftConfig из: /content/results_rugpt3small_lora_qa_epoch1_lr1e-4_v2/final_lora_adapters
PeftConfig успешно загружен.

Загрузка базовой модели 'sberbank-ai/rugpt3small_based_on_gpt2' с квантованием для инференса...
Базовая модель успешно загружена с квантованием.
Загрузка и применение LoRA адаптеров из /content/results_rugpt3small_lora_qa_epoch1_lr1e-4_v2/final_lora_adapters к базовой модели...
Модель с LoRA адаптерами готова для инференса.

--- Тестирование модели ---
Используются адаптеры из: /content/results_rugpt3small_lora_qa_epoch1_lr1e-4_v2/final_lora_adapters
Используемые параметры генерации: {'max_new_tokens': 150, 'temperature': 0.1, 'top_p': 0.9, 'repetition_penalty': 1.15, 'do_sample': True}

Вопрос: Кто по образованию Балакина Татьяна Петровна?
Ответ модели: По образованию Балакин Дмитрий Петрович, который учился в школе № 1.

Вопрос: Какие направления в образовании упомянуты в контексте обсуждения 

# **Конфигурация LoRA - Модифицированная**

In [ ]:
#########################_Проверка_мол_все_ли_у_нас_хорошо_#########################

if 'model' not in globals() or 'tokenizer' not in globals():
    print("WARNING: Базовая модель или токенизатор не загружены")
    raise RuntimeError("Базовая модель или токенизатор не загружены")
elif 'train_dataset' not in globals() or 'eval_dataset' not in globals():
    print("WARNING: Обучающий или валидационный датасет не готовы")
else:
    print("Базовая модель, токенизатор и датасеты (предположительно) готовы.")


#########################_Подготовка_модели_для_k-битного обучения_(если используется квантование)_#########################

    try:
        if hasattr(model, 'is_loaded_in_4bit') or hasattr(model, 'is_loaded_in_8bit'):
             if not getattr(model, 'is_prepared_for_kbit_training', False):
                print("Подготовка модели для k-bit training (QLoRA)...")
                model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True) # Явно включаем checkpointing
                model.is_prepared_for_kbit_training = True # Ставим флаг
                print("Модель подготовлена для k-bit training (QLoRA).")
             else:
                print("Модель уже была подготовлена для k-bit training.")
        else:
            print("Модель не загружена с квантованием, prepare_model_for_kbit_training не будет применен.")

    except Exception as e:
        print(f"Ошибка при prepare_model_for_kbit_training: {e}")
        print("Продолжаем без явной подготовки, но могут быть проблемы со стабильностью QLoRA, если она не была сделана ранее.")

#########################_Определение_НОВОЙ_конфигурации_LoRA_#########################

    # вынес гиперпараметры
    lora_r = 32
    lora_alpha = lora_r * 2
    lora_dropout = 0.05
    # Для GPT-2 архитектуры (rugpt3small):
    # 'c_attn' - attention QKV linear layer
    # 'c_proj' - attention output projection linear layer
    # 'c_fc'   - first linear layer in MLP
    # 'c_proj' in MLP (второй линейный слой) тоже можно, но начнем с этих трех типов
    lora_target_modules = ["c_attn", "c_proj", "c_fc"]

    lora_config_new = LoraConfig(
        r=lora_r,
        lora_alpha=lora_alpha,
        target_modules=lora_target_modules,
        lora_dropout=lora_dropout,
        bias="none",
        task_type="CAUSAL_LM"
    )
    print(f"\nНовая LoraConfig создана: r={lora_r}, alpha={lora_alpha}, target_modules={lora_target_modules}")
    print(lora_config_new)


#########################_Применение_LoRA_к_модели_#########################

    # Важно: 'model' здесь должна быть базовой моделью, а не уже обернутой в PeftModel

    # Проверка, не является ли 'model' уже PeftModel
    if isinstance(model, PeftModel):
        print("ПРЕДУПРЕЖДЕНИЕ: 'model' уже является PeftModel. Попытка получить базовую модель...")
        base_model_for_lora = model.base_model.model
    else:
        base_model_for_lora = model

    try:
        model_to_train = get_peft_model(base_model_for_lora, lora_config_new)
        print("\nНовая конфигурация LoRA применена к модели.")

        print("\nИнформация о параметрах после применения НОВОЙ LoRA:")
        model_to_train.print_trainable_parameters()

        print("\nМодель 'model_to_train' готова к обучению с новой LoRA конфигурацией.")

    except Exception as e:
        print(f"Ошибка при применении новой LoRA к модели: {e}")
        model_to_train = None

Базовая модель, токенизатор и датасеты (предположительно) готовы.
Подготовка модели для k-bit training (QLoRA)...
Модель подготовлена для k-bit training (QLoRA).

Новая LoraConfig создана: r=32, alpha=64, target_modules=['c_attn', 'c_proj', 'c_fc']
LoraConfig(task_type='CAUSAL_LM', peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, inference_mode=False, r=32, target_modules={'c_attn', 'c_fc', 'c_proj'}, exclude_modules=None, lora_alpha=64, lora_dropout=0.05, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False)

Новая конфигурация LoRA применена к модели.

Информ

In [ ]:
#########################_Проверка_мол_все_ли_у_нас_хорошо_#########################

if 'model_to_train' not in globals() or not model_to_train or not hasattr(model_to_train, 'peft_config'):
    print("Warning: Модель для обучения 'model_to_train' (с PEFT конфигурацией) не определена или некорректна")
    raise RuntimeError("'model_to_train' не готова, принудительное прерывание.")
elif model_to_train.peft_config['default'].r != 32: # Проверка, что r=32
     print(f"ПРЕДУПРЕЖДЕНИЕ: r в model_to_train.peft_config ({model_to_train.peft_config['default'].r}) "
           f"не соответствует ожидаемому (32). Убедитесь, что вы применили новую LoRA конфигурацию.")
elif 'train_dataset' not in globals() or 'eval_dataset' not in globals():
    print("WARNING: train/val датасет не готовы")
    raise RuntimeError("train_dataset или eval_dataset не определены, принудительное прерывание.")
elif 'tokenizer' not in globals():
    print("WARNING: Токенизатор не загружен")
    raise RuntimeError("tokenizer не определен. Прерывание.")
else:
    print("Все компоненты для обучения (с новой LoRA конфигурацией) готовы")

    # Новая директория для сохранения результатов этого эксперимента
    base_output_dir_new_lora = "./results_rugpt3small_lora_r32_targets_extended"
    final_lora_adapters_path_new_lora = os.path.join(base_output_dir_new_lora, "final_lora_adapters")

    print(f"Результаты (чекпоинты) будут в: {base_output_dir_new_lora}")
    print(f"Финальные адаптеры будут сохранены в: {final_lora_adapters_path_new_lora}")

    training_args = TrainingArguments(
        output_dir=base_output_dir_new_lora,
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        num_train_epochs=1,
        learning_rate=1e-4, # Оставляем этот LR, так как он давал лучший eval_loss

        logging_strategy="steps",
        logging_steps=50,

        save_strategy="steps",
        save_steps=100,

        eval_strategy="steps",
        eval_steps=100,

        do_eval=True,

        save_total_limit=2,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,

        report_to="none",
        fp16=True,
    )

    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

    trainer = Trainer(
        model=model_to_train,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
    )

    print("\nTrainer сконфигурирован. Начинаем обучение...")

    try:
        train_result = trainer.train()
        print("\nОбучение завершено!")

        os.makedirs(final_lora_adapters_path_new_lora, exist_ok=True)
        trainer.save_model(final_lora_adapters_path_new_lora)
        print(f"Финальные LoRA адаптеры сохранены в: {final_lora_adapters_path_new_lora}")

        metrics = train_result.metrics
        print("\nМетрики обучения:")
        print(metrics)
        print("\nЛог обучения (история):")
        for log_entry in trainer.state.log_history:
            print(log_entry)

    except Exception as e:
        if "CUDA out of memory" in str(e):
            print("\nОШИБКА CUDA OUT OF MEMORY!")
            print("Попробуйте уменьшить 'per_device_train_batch_size' (например, до 2 или 1) и перезапустить обучение.")
        else:
            print(f"\nПроизошла ошибка во время обучения: {e}")
        import traceback
        traceback.print_exc()

<ipython-input-29-f92f0663e17b>:59: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Все компоненты для обучения (с новой LoRA конфигурацией) готовы.
Результаты (чекпоинты) будут в: ./results_rugpt3small_lora_r32_targets_extended
Финальные адаптеры будут сохранены в: ./results_rugpt3small_lora_r32_targets_extended/final_lora_adapters

Trainer сконфигурирован. Начинаем обучение...


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss
100,3.157800,2.635995
200,2.648300,2.437575
300,2.534400,2.363811
400,2.477100,2.329300


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/


Обучение завершено!
Финальные LoRA адаптеры сохранены в: ./results_rugpt3small_lora_r32_targets_extended/final_lora_adapters

Метрики обучения:
{'train_runtime': 740.4349, 'train_samples_per_second': 10.502, 'train_steps_per_second': 0.656, 'total_flos': 2144523962548224.0, 'train_loss': 2.915943381227093, 'epoch': 1.0}

Лог обучения (история):
{'loss': 5.5159, 'grad_norm': 4.073153018951416, 'learning_rate': 9.012345679012346e-05, 'epoch': 0.102880658436214, 'step': 50}
{'loss': 3.1578, 'grad_norm': 1.8507589101791382, 'learning_rate': 7.983539094650207e-05, 'epoch': 0.205761316872428, 'step': 100}
{'eval_loss': 2.6359951496124268, 'eval_runtime': 21.1477, 'eval_samples_per_second': 40.903, 'eval_steps_per_second': 5.154, 'epoch': 0.205761316872428, 'step': 100}
{'loss': 2.7411, 'grad_norm': 2.129037380218506, 'learning_rate': 6.954732510288067e-05, 'epoch': 0.30864197530864196, 'step': 150}
{'loss': 2.6483, 'grad_norm': 1.7386090755462646, 'learning_rate': 5.925925925925926e-05, 'ep

# **Тестирование и Инференс - для r=32, расширенные таргеты**

In [ ]:
lora_adapters_path = "./results_rugpt3small_lora_r32_targets_extended/final_lora_adapters"
base_model_name_for_inference = "sberbank-ai/rugpt3small_based_on_gpt2"

# Используем "Набор 1" как наиболее сбалансированный из предыдущих тестов
generation_params_set = {
    "max_new_tokens": 150,
    "temperature": 0.5,
    "top_p": 0.9,
    "repetition_penalty": 1.15,
    "do_sample": True
}
current_generation_params = generation_params_set
print(f"Загрузка токенизатора для: {base_model_name_for_inference}")


try:
    tokenizer = AutoTokenizer.from_pretrained(base_model_name_for_inference)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        print(f"Установлен pad_token: {tokenizer.pad_token} (ID: {tokenizer.pad_token_id})")
except Exception as e:
    print(f"Ошибка при загрузке токенизатора: {e}")
    tokenizer = None

if not os.path.exists(lora_adapters_path):
    print(f"ОШИБКА: Путь к адаптерам не найден: {lora_adapters_path}")
    config = None
    actual_base_model_name = None
elif tokenizer:
    try:
        print(f"\nЗагрузка PeftConfig из: {lora_adapters_path}")
        config = PeftConfig.from_pretrained(lora_adapters_path)
        if config.base_model_name_or_path.lower() != base_model_name_for_inference.lower():
            print(f"ПРЕДУПРЕЖДЕНИЕ: Имя базовой модели в PeftConfig ({config.base_model_name_or_path}) "
                  f"не совпадает с ожидаемым ({base_model_name_for_inference}). Используем имя из PeftConfig.")
        actual_base_model_name = config.base_model_name_or_path
        print("PeftConfig успешно загружен.")
    except Exception as e:
        print(f"Ошибка при загрузке PeftConfig: {e}")
        config = None
        actual_base_model_name = None
else:
    config = None
    actual_base_model_name = None


final_model_for_inference = None
if config and actual_base_model_name:
    bnb_config_inference = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=False,
    )

    print(f"\nЗагрузка базовой модели '{actual_base_model_name}' с квантованием для инференса...")
    try:
        base_model = AutoModelForCausalLM.from_pretrained(
            actual_base_model_name,
            quantization_config=bnb_config_inference,
            device_map="auto",
            trust_remote_code=True
        )
        print("Базовая модель успешно загружена с квантованием.")

        print(f"Загрузка и применение LoRA адаптеров из {lora_adapters_path} к базовой модели...")
        final_model_for_inference = PeftModel.from_pretrained(base_model, lora_adapters_path)
        final_model_for_inference.eval()
        print("Модель с LoRA адаптерами готова для инференса.")

    except Exception as e:
        print(f"Ошибка при загрузке базовой модели или применении LoRA: {e}")
        final_model_for_inference = None
else:
    if not tokenizer:
        print("Токенизатор не был загружен, инференс невозможен.")
    elif not os.path.exists(lora_adapters_path):
        pass
    elif not config:
         print("PeftConfig не был загружен, инференс невозможен.")
    elif not actual_base_model_name:
        print("Не удалось определить имя базовой модели из PeftConfig.")


if final_model_for_inference and tokenizer:
    def generate_answer(question, model, tokenizer, gen_params):
        prompt = f"Инструкция: {question}\n\nОтвет:"
        max_input_length = 512 - gen_params.get("max_new_tokens", 100)
        inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=max_input_length)
        inputs = {k: v.to(next(model.parameters()).device) for k, v in inputs.items()}

        generation_call_params = {**inputs, **gen_params}
        generation_call_params['pad_token_id'] = tokenizer.pad_token_id
        generation_call_params['eos_token_id'] = tokenizer.eos_token_id

        with torch.no_grad():
            outputs = model.generate(**generation_call_params)

        answer_tokens = outputs[0][inputs['input_ids'].shape[1]:]
        answer_text = tokenizer.decode(answer_tokens, skip_special_tokens=True)

        processed_answer = answer_text.strip()
        while processed_answer.lower().startswith("ответ:"):
            processed_answer = processed_answer[len("ответ:"):]
            processed_answer = processed_answer.strip()

        return processed_answer

    test_questions = [
        "Кто по образованию Балакина Татьяна Петровна?",
        "Какие направления в образовании упомянуты в контексте обсуждения с Татьяной Петровной?",
        "Почему важно участвовать в олимпиадах и хакатонах по мнению Татьяны Петровны?"
    ]

    print("\n--- Тестирование модели ---")
    print(f"Используются адаптеры из: {lora_adapters_path}")
    print(f"Используемые параметры генерации: {current_generation_params}")
    for q in test_questions:
        print(f"\nВопрос: {q}")
        answer = generate_answer(q, final_model_for_inference, tokenizer, current_generation_params)
        print(f"Ответ модели: {answer}")
else:
    print("\nМодель для инференса не была загружена или токенизатор отсутствует. Проверьте ошибки выше.")

Загрузка токенизатора для: sberbank-ai/rugpt3small_based_on_gpt2

Загрузка PeftConfig из: ./results_rugpt3small_lora_r32_targets_extended/final_lora_adapters
PeftConfig успешно загружен.

Загрузка базовой модели 'sberbank-ai/rugpt3small_based_on_gpt2' с квантованием для инференса...
Базовая модель успешно загружена с квантованием.
Загрузка и применение LoRA адаптеров из ./results_rugpt3small_lora_r32_targets_extended/final_lora_adapters к базовой модели...
Модель с LoRA адаптерами готова для инференса.

--- Тестирование модели ---
Используются адаптеры из: ./results_rugpt3small_lora_r32_targets_extended/final_lora_adapters
Используемые параметры генерации: {'max_new_tokens': 150, 'temperature': 0.5, 'top_p': 0.9, 'repetition_penalty': 1.15, 'do_sample': True}

Вопрос: Кто по образованию Балакина Татьяна Петровна?
Ответ модели: В Википедии указано, что Балакин Татьяна Петровна — преподаватель математики. Она преподает математику в школе № 1. Это не соответствует действительности.

* * *

# **rugpt3medium**

In [ ]:
model_name = "sberbank-ai/rugpt3medium_based_on_gpt2"
model = None
tokenizer = None

print(f"Попытка загрузки базовой модели: {model_name}")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16, # float16 для T4
    bnb_4bit_use_double_quant=False,
)

try:
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True # Для моделей Сбера обычно нужно
    )
    print(f"Модель '{model_name}' успешно загружена с квантованием.")
    print(f"  Тип данных модели на устройстве: {model.dtype if model else 'N/A'}")
    if model and hasattr(next(model.parameters(), None), 'device'):
         print(f"  Устройство первого параметра: {next(model.parameters()).device}")

    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    if tokenizer.pad_token is None:
        print("pad_token is None. Setting pad_token = eos_token.")
        tokenizer.pad_token = tokenizer.eos_token
        if model.config.pad_token_id is None and tokenizer.pad_token_id is not None:
             model.config.pad_token_id = tokenizer.pad_token_id
    else:
        print(f"pad_token уже установлен: {tokenizer.pad_token} (ID: {tokenizer.pad_token_id})")

    print("\nБазовая модель и токенизатор загружены и готовы.")

except Exception as e:
    print(f"Произошла ошибка при загрузке модели или токенизатора: {e}")
    model = None
    tokenizer = None
    import traceback
    traceback.print_exc()

if model:
    print("\nИнформация о модели (если загружена):")
    try:
        print(f"  Примерный размер в памяти (оценка): {model.get_memory_footprint() / (1024*1024):.2f} MB")
    except:
         print(f"  Количество параметров (исходных): {model.config.num_parameters() / 1e6:.2f}M")
    print(f"  Конфигурация модели: {model.config.__class__.__name__}")

Попытка загрузки базовой модели: sberbank-ai/rugpt3medium_based_on_gpt2


config.json:   0%|          | 0.00/761 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.73G [00:00<?, ?B/s]

Модель 'sberbank-ai/rugpt3medium_based_on_gpt2' успешно загружена с квантованием.
  Тип данных модели на устройстве: torch.float16
  Устройство первого параметра: cuda:0


tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.61M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

pad_token уже установлен: <pad> (ID: 0)

Базовая модель и токенизатор загружены и готовы.

Информация о модели (если загружена):
  Примерный размер в памяти (оценка): 342.77 MB
  Конфигурация модели: GPT2Config


# **Конфигурация LoRA для rugpt3medium**

In [ ]:
#########################_Проверка_мол_все_ли_у_нас_хорошо_#########################

if 'model' not in globals() or model is None or 'tokenizer' not in globals() or tokenizer is None:
    print("WARNING: Базовая модель или токенизатор не загружены")
    raise RuntimeError("Базовая модель или токенизатор не загружены")
else:
    print("Базовая модель и токенизатор (предположительно rugpt3medium) готовы для LoRA.")

    try:
        if hasattr(model, 'is_loaded_in_4bit') or hasattr(model, 'is_loaded_in_8bit'):
             if not getattr(model, 'is_prepared_for_kbit_training', False):
                print("Подготовка модели для k-bit training (QLoRA)")
                model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)
                model.is_prepared_for_kbit_training = True
                print("Модель подготовлена для k-bit training (QLoRA).")
             else:
                print("Модель уже была подготовлена для k-bit training.")
        else:
            print("Модель не загружена с квантованием, prepare_model_for_kbit_training не будет применен")

    except Exception as e:
        print(f"Ошибка при prepare_model_for_kbit_training: {e}")

#########################_# LoRA_конфигурация_(начнем_с_r=16_для_rugpt3medium)_#########################

    # Вынесем все параметры
    lora_r = 16
    lora_alpha = lora_r * 2
    lora_dropout = 0.05
    # Target modules для GPT-2 архитектуры (rugpt3medium такая же)
    lora_target_modules = ["c_attn", "c_proj", "c_fc"]

    lora_config_medium = LoraConfig(
        r=lora_r,
        lora_alpha=lora_alpha,
        target_modules=lora_target_modules,
        lora_dropout=lora_dropout,
        bias="none",
        task_type="CAUSAL_LM"
    )
    print(f"\nLoraConfig для rugpt3medium: r={lora_r}, alpha={lora_alpha}, target_modules={lora_target_modules}")
    print(lora_config_medium)

    base_model_for_lora = model

    try:
        model_to_train = get_peft_model(base_model_for_lora, lora_config_medium)
        print("\nLoRA применена к модели rugpt3medium.")
        print("\nИнформация о параметрах после применения LoRA:")
        model_to_train.print_trainable_parameters()
        print("\nМодель 'model_to_train' готова к обучению.")
    except Exception as e:
        print(f"Ошибка при применении LoRA к модели: {e}")
        model_to_train = None

Базовая модель и токенизатор (предположительно rugpt3medium) готовы для LoRA.
Подготовка модели для k-bit training (QLoRA)...
Модель подготовлена для k-bit training (QLoRA).

LoraConfig для rugpt3medium: r=16, alpha=32, target_modules=['c_attn', 'c_proj', 'c_fc']
LoraConfig(task_type='CAUSAL_LM', peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, inference_mode=False, r=16, target_modules={'c_attn', 'c_fc', 'c_proj'}, exclude_modules=None, lora_alpha=32, lora_dropout=0.05, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False)

LoRA применена к модели rugpt3medium

# **Обучение rugpt3medium с LoRA**

In [ ]:
#########################_Проверка_мол_все_ли_у_нас_хорошо_########################

if 'model_to_train' not in globals() or not model_to_train or not hasattr(model_to_train, 'peft_config'):
    print("WARNING: Модель для обучения 'model_to_train' (с PEFT) не готова.")
    raise RuntimeError("'model_to_train' не готова.")
elif 'train_dataset' not in globals() or 'eval_dataset' not in globals():
    print("WARNING: Датасеты не готовы.")
    raise RuntimeError("Датасеты не готовы.")
elif 'tokenizer' not in globals() or tokenizer is None:
    print("Ошибка: Токенизатор не загружен")
    raise RuntimeError("Токенизатор не загружен.")
else:
    print("Все компоненты для обучения rugpt3medium с LoRA готовы.")

    output_dir_medium = "./results_rugpt3medium_lora_r16_epoch1_lr1e-4"
    final_adapters_path_medium = os.path.join(output_dir_medium, "final_lora_adapters")

    print(f"Результаты (чекпоинты) будут в: {output_dir_medium}")
    print(f"Финальные адаптеры будут сохранены в: {final_adapters_path_medium}")

    training_args = TrainingArguments(
        output_dir=output_dir_medium,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=16,
        num_train_epochs=1,
        learning_rate=1e-4,
        weight_decay=0.1,
        logging_strategy="steps",
        logging_steps=25,
        save_strategy="steps",
        save_steps=50,
        eval_strategy="steps",
        eval_steps=50,
        do_eval=True,
        save_total_limit=2,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        report_to="none",
        fp16=True,
    )

    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

    trainer = Trainer(
        model=model_to_train,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
    )

    print("\nTrainer сконфигурирован для rugpt3medium. Начинаем обучение")

    try:
        train_result = trainer.train()
        print("\nОбучение завершено!")

        os.makedirs(final_adapters_path_medium, exist_ok=True)
        trainer.save_model(final_adapters_path_medium)
        print(f"Финальные LoRA адаптеры сохранены в: {final_adapters_path_medium}")

        metrics = train_result.metrics
        print("\nМетрики обучения:")
        print(metrics)
        print("\nЛог обучения (история):")
        for log_entry in trainer.state.log_history:
            print(log_entry)

    except Exception as e:
        if "CUDA out of memory" in str(e):
            print("\nОШИБКА CUDA OUT OF MEMORY!")
            print("Это ожидаемо для rugpt3medium. Убедитесь, что 'per_device_train_batch_size' равен 1.")
            print("Если все равно ошибка, возможно, T4 не хватает памяти даже для batch_size=1 с этой LoRA конфигурацией.")
            print("Можно попробовать QLoRA с 8-bit загрузкой (load_in_8bit=True в bnb_config), но это менее эффективно.")
        else:
            print(f"\nПроизошла ошибка во время обучения: {e}")
        import traceback
        traceback.print_exc()

# **Тестирование rugpt3medium с LoRA**

In [ ]:
lora_adapters_path = "./results_rugpt3medium_lora_r16_epoch1_lr1e-4/final_lora_adapters"
base_model_name_for_inference = "sberbank-ai/rugpt3medium_based_on_gpt2" # <--- БАЗОВАЯ МОДЕЛЬ MEDIUM

generation_params_set = {
    "max_new_tokens": 150,
    "temperature": 0.5,
    "top_p": 0.9,
    "repetition_penalty": 1.15,
    "do_sample": True
}
current_generation_params = generation_params_set

print(f"Загрузка токенизатора для: {base_model_name_for_inference}")
try:
    tokenizer = AutoTokenizer.from_pretrained(base_model_name_for_inference, trust_remote_code=True) # Добавил trust_remote_code
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        print(f"Установлен pad_token: {tokenizer.pad_token} (ID: {tokenizer.pad_token_id})")
except Exception as e:
    print(f"Ошибка при загрузке токенизатора: {e}")
    tokenizer = None

if not os.path.exists(lora_adapters_path):
    print(f"ОШИБКА: Путь к адаптерам не найден: {lora_adapters_path}")
    config = None
    actual_base_model_name = None
elif tokenizer:
    try:
        print(f"\nЗагрузка PeftConfig из: {lora_adapters_path}")
        config = PeftConfig.from_pretrained(lora_adapters_path)
        if config.base_model_name_or_path.lower() != base_model_name_for_inference.lower():
            print(f"ПРЕДУПРЕЖДЕНИЕ: Имя базовой модели в PeftConfig ({config.base_model_name_or_path}) "
                  f"не совпадает с ожидаемым ({base_model_name_for_inference}). Используем имя из PeftConfig.")
        actual_base_model_name = config.base_model_name_or_path
        print("PeftConfig успешно загружен.")
    except Exception as e:
        print(f"Ошибка при загрузке PeftConfig: {e}")
        config = None
        actual_base_model_name = None
else:
    config = None
    actual_base_model_name = None


final_model_for_inference = None
if config and actual_base_model_name:
    bnb_config_inference = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=False,
    )

    print(f"\nЗагрузка базовой модели '{actual_base_model_name}' с квантованием для инференса...")
    try:
        base_model_for_peft = AutoModelForCausalLM.from_pretrained( # Используем другое имя переменной
            actual_base_model_name,
            quantization_config=bnb_config_inference,
            device_map="auto",
            trust_remote_code=True
        )
        print("Базовая модель успешно загружена с квантованием.")

        print(f"Загрузка и применение LoRA адаптеров из {lora_adapters_path} к базовой модели...")
        final_model_for_inference = PeftModel.from_pretrained(base_model_for_peft, lora_adapters_path) # Передаем base_model_for_peft
        final_model_for_inference.eval()
        print("Модель с LoRA адаптерами готова для инференса.")

    except Exception as e:
        print(f"Ошибка при загрузке базовой модели или применении LoRA: {e}")
        final_model_for_inference = None
else:
    if not tokenizer:
        print("Токенизатор не был загружен, инференс невозможен.")
    elif not os.path.exists(lora_adapters_path):
        pass
    elif not config:
         print("PeftConfig не был загружен, инференс невозможен.")
    elif not actual_base_model_name:
        print("Не удалось определить имя базовой модели из PeftConfig.")


if final_model_for_inference and tokenizer:
    def generate_answer(question, model, tokenizer, gen_params):
        prompt = f"Инструкция: {question}\n\nОтвет:"
        max_input_length = 512 - gen_params.get("max_new_tokens", 100)
        inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=max_input_length)
        inputs = {k: v.to(next(model.parameters()).device) for k, v in inputs.items()}

        generation_call_params = {**inputs, **gen_params}
        generation_call_params['pad_token_id'] = tokenizer.pad_token_id
        generation_call_params['eos_token_id'] = tokenizer.eos_token_id

        with torch.no_grad():
            outputs = model.generate(**generation_call_params)

        answer_tokens = outputs[0][inputs['input_ids'].shape[1]:]
        answer_text = tokenizer.decode(answer_tokens, skip_special_tokens=True)

        processed_answer = answer_text.strip()
        while processed_answer.lower().startswith("ответ:"):
            processed_answer = processed_answer[len("ответ:"):]
            processed_answer = processed_answer.strip()

        return processed_answer

    test_questions = [
        "Кто по образованию Балакина Татьяна Петровна?",
        "Какие направления в образовании упомянуты в контексте обсуждения с Татьяной Петровной?",
        "Почему важно участвовать в олимпиадах и хакатонах по мнению Татьяны Петровны?"
    ]

    print("\n--- Тестирование модели ---")
    print(f"Используются адаптеры из: {lora_adapters_path}")
    print(f"Используемые параметры генерации: {current_generation_params}")
    for q in test_questions:
        print(f"\nВопрос: {q}")
        answer = generate_answer(q, final_model_for_inference, tokenizer, current_generation_params)
        print(f"Ответ модели: {answer}")
else:
    print("\nМодель для инференса не была загружена или токенизатор отсутствует. Проверьте ошибки выше.")

Загрузка токенизатора для: sberbank-ai/rugpt3medium_based_on_gpt2

Загрузка PeftConfig из: ./results_rugpt3medium_lora_r16_epoch1_lr1e-4/final_lora_adapters
PeftConfig успешно загружен.

Загрузка базовой модели 'sberbank-ai/rugpt3medium_based_on_gpt2' с квантованием для инференса...
Базовая модель успешно загружена с квантованием.
Загрузка и применение LoRA адаптеров из ./results_rugpt3medium_lora_r16_epoch1_lr1e-4/final_lora_adapters к базовой модели...
Модель с LoRA адаптерами готова для инференса.

--- Тестирование модели ---
Используются адаптеры из: ./results_rugpt3medium_lora_r16_epoch1_lr1e-4/final_lora_adapters
Используемые параметры генерации: {'max_new_tokens': 150, 'temperature': 0.5, 'top_p': 0.9, 'repetition_penalty': 1.15, 'do_sample': True}

Вопрос: Кто по образованию Балакина Татьяна Петровна?
Ответ модели: Татьяна Петровна Балакина - экономист, кандидат экономических наук. Она окончила факультет менеджмента в Московском государственном университете имени М.В. Ломоносов

# **Тестирование rugpt3medium с LoRA**

In [ ]:
lora_adapters_path = "./results_rugpt3medium_lora_r16_epoch1_lr1e-4/final_lora_adapters" # <--- ПУТЬ К АДАПТЕРАМ MEDIUM
base_model_name_for_inference = "sberbank-ai/rugpt3medium_based_on_gpt2" # <--- БАЗОВАЯ МОДЕЛЬ MEDIUM

generation_params_set = {
    "max_new_tokens": 200,
    "temperature": 0.2,
    "top_p": 0.9,
    "repetition_penalty": 1.1,
    "do_sample": True
}
current_generation_params = generation_params_set

print(f"Загрузка токенизатора для: {base_model_name_for_inference}")
try:
    tokenizer = AutoTokenizer.from_pretrained(base_model_name_for_inference, trust_remote_code=True) # Добавил trust_remote_code
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        print(f"Установлен pad_token: {tokenizer.pad_token} (ID: {tokenizer.pad_token_id})")
except Exception as e:
    print(f"Ошибка при загрузке токенизатора: {e}")
    tokenizer = None

if not os.path.exists(lora_adapters_path):
    print(f"ОШИБКА: Путь к адаптерам не найден: {lora_adapters_path}")
    config = None
    actual_base_model_name = None
elif tokenizer:
    try:
        print(f"\nЗагрузка PeftConfig из: {lora_adapters_path}")
        config = PeftConfig.from_pretrained(lora_adapters_path)
        if config.base_model_name_or_path.lower() != base_model_name_for_inference.lower():
            print(f"ПРЕДУПРЕЖДЕНИЕ: Имя базовой модели в PeftConfig ({config.base_model_name_or_path}) "
                  f"не совпадает с ожидаемым ({base_model_name_for_inference}). Используем имя из PeftConfig.")
        actual_base_model_name = config.base_model_name_or_path
        print("PeftConfig успешно загружен.")
    except Exception as e:
        print(f"Ошибка при загрузке PeftConfig: {e}")
        config = None
        actual_base_model_name = None
else:
    config = None
    actual_base_model_name = None


final_model_for_inference = None
if config and actual_base_model_name:
    bnb_config_inference = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=False,
    )

    print(f"\nЗагрузка базовой модели '{actual_base_model_name}' с квантованием для инференса...")
    try:
        base_model_for_peft = AutoModelForCausalLM.from_pretrained(
            actual_base_model_name,
            quantization_config=bnb_config_inference,
            device_map="auto",
            trust_remote_code=True
        )
        print("Базовая модель успешно загружена с квантованием.")

        print(f"Загрузка и применение LoRA адаптеров из {lora_adapters_path} к базовой модели...")
        final_model_for_inference = PeftModel.from_pretrained(base_model_for_peft, lora_adapters_path) # Передаем base_model_for_peft
        final_model_for_inference.eval()
        print("Модель с LoRA адаптерами готова для инференса.")

    except Exception as e:
        print(f"Ошибка при загрузке базовой модели или применении LoRA: {e}")
        final_model_for_inference = None
else:
    if not tokenizer:
        print("Токенизатор не был загружен, инференс невозможен.")
    elif not os.path.exists(lora_adapters_path):
        pass
    elif not config:
         print("PeftConfig не был загружен, инференс невозможен.")
    elif not actual_base_model_name:
        print("Не удалось определить имя базовой модели из PeftConfig.")


if final_model_for_inference and tokenizer:
    def generate_answer(question, model, tokenizer, gen_params):
        prompt = f"Инструкция: {question}\n\nОтвет:"
        max_input_length = 512 - gen_params.get("max_new_tokens", 100)
        inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=max_input_length)
        inputs = {k: v.to(next(model.parameters()).device) for k, v in inputs.items()}

        generation_call_params = {**inputs, **gen_params}
        generation_call_params['pad_token_id'] = tokenizer.pad_token_id
        generation_call_params['eos_token_id'] = tokenizer.eos_token_id

        with torch.no_grad():
            outputs = model.generate(**generation_call_params)

        answer_tokens = outputs[0][inputs['input_ids'].shape[1]:]
        answer_text = tokenizer.decode(answer_tokens, skip_special_tokens=True)

        processed_answer = answer_text.strip()
        while processed_answer.lower().startswith("ответ:"):
            processed_answer = processed_answer[len("ответ:"):]
            processed_answer = processed_answer.strip()

        return processed_answer

    test_questions = [
        "Кто по образованию Балакина Татьяна Петровна?",
        "Какие направления в образовании упомянуты в контексте обсуждения с Татьяной Петровной?",
        "Почему важно участвовать в олимпиадах и хакатонах по мнению Татьяны Петровны?"
    ]

    print("\n--- Тестирование модели ---")
    print(f"Используются адаптеры из: {lora_adapters_path}")
    print(f"Используемые параметры генерации: {current_generation_params}")
    for q in test_questions:
        print(f"\nВопрос: {q}")
        answer = generate_answer(q, final_model_for_inference, tokenizer, current_generation_params)
        print(f"Ответ модели: {answer}")
else:
    print("\nМодель для инференса не была загружена или токенизатор отсутствует. Проверьте ошибки выше.")

Загрузка токенизатора для: sberbank-ai/rugpt3medium_based_on_gpt2

Загрузка PeftConfig из: ./results_rugpt3medium_lora_r16_epoch1_lr1e-4/final_lora_adapters
PeftConfig успешно загружен.

Загрузка базовой модели 'sberbank-ai/rugpt3medium_based_on_gpt2' с квантованием для инференса...
Базовая модель успешно загружена с квантованием.
Загрузка и применение LoRA адаптеров из ./results_rugpt3medium_lora_r16_epoch1_lr1e-4/final_lora_adapters к базовой модели...
Модель с LoRA адаптерами готова для инференса.

--- Тестирование модели ---
Используются адаптеры из: ./results_rugpt3medium_lora_r16_epoch1_lr1e-4/final_lora_adapters
Используемые параметры генерации: {'max_new_tokens': 200, 'temperature': 0.2, 'top_p': 0.9, 'repetition_penalty': 1.1, 'do_sample': True}

Вопрос: Кто по образованию Балакина Татьяна Петровна?
Ответ модели: Татьяна Петровна Балакина - кандидат экономических наук, доцент кафедры экономики и управления в образовании МФТИ. Она окончила факультет прикладной математики и инфо